In [34]:
import os
GOOGLE_APPLICATION_CREDENTIALS = "/Users/zacharynguyen/Documents/GitHub/2024/End-to-End-Vertex-AI-Pipeline-for-Fraud-Detection/key/e2e-fraud-detection-debf1c9863af.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_APPLICATION_CREDENTIALS

In [35]:
PROJECT_ID = 'e2e-fraud-detection'
REGION = 'us-central1'
EXPERIMENT = 'Predictions with Vertex AI Endpoint'
SERIES = 'bqml'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud_dataset'
BQ_TABLE = 'prepped-data'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

In [36]:
from google.cloud import bigquery
from google.cloud import aiplatform
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np

from google.api import httpbody_pb2
import json

In [37]:
bq = bigquery.Client(project = PROJECT_ID)
aiplatform.init(project = PROJECT_ID, location = REGION)

In [38]:
query = f"""
SELECT * EXCEPT({VAR_TARGET}, {VAR_OMIT}, splits)
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
WHERE splits='TEST'
LIMIT {n}
"""

print(query)


SELECT * EXCEPT(Class, transaction_id, splits)
FROM `e2e-fraud-detection.fraud_dataset.prepped-data`
WHERE splits='TEST'
LIMIT 10


In [39]:
n = 10
pred = bq.query(
    query = query
).to_dataframe()

In [40]:
pred

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,140315.0,1.882751,0.532175,0.150708,3.889469,0.011037,0.101617,-0.105079,-0.027678,-0.649422,...,-0.225558,-0.401006,-1.015581,0.487635,-0.024250,-0.512936,-0.516050,0.011179,-0.026608,0.0
1,171011.0,-1.345222,2.280635,2.149265,4.465909,-0.029695,2.007951,-2.101858,-5.197115,-2.235785,...,1.594403,-2.323161,1.696769,-0.201321,0.007053,0.055551,0.580323,-0.086456,-0.050837,0.0
2,35734.0,-2.571353,-2.286027,2.909699,-0.264519,2.295561,0.264278,-1.412284,0.431003,1.369659,...,-0.258066,0.061358,0.799850,-0.363365,-0.520853,-0.024029,1.180507,-0.309633,-0.296332,0.0
3,12973.0,1.042687,0.224056,1.651554,2.756488,-0.541425,0.970706,-0.875859,0.406813,1.234202,...,-0.271378,-0.165518,-0.108077,0.082059,-0.072716,0.148948,-0.066969,0.024569,0.014142,0.0
4,64268.0,1.316193,-0.009926,-0.137555,0.587167,-0.308887,-0.799481,0.042878,-0.080183,0.612074,...,-0.288865,-0.133028,-0.349540,-0.136149,-0.126894,0.639381,0.595106,-0.070506,-0.010366,0.0
5,119592.0,2.139741,0.245651,-2.654856,0.178287,1.336991,-0.724664,0.906032,-0.436125,-0.528015,...,-0.160988,0.296681,1.036285,-0.216033,0.345316,0.747103,0.700184,-0.123739,-0.099989,0.0
6,123071.0,1.847235,0.339967,-1.522324,3.423434,1.429797,1.497229,0.096091,0.412087,-1.069708,...,-0.517461,0.328189,1.051528,-0.047523,-1.664775,0.201577,0.413007,-0.029553,-0.096775,0.0
7,128494.0,2.096836,-0.084084,-1.622199,0.479912,0.034264,-1.448540,0.403478,-0.382239,0.605741,...,-0.344547,-0.054720,-0.001575,0.100703,0.066990,0.129055,0.563681,-0.105465,-0.080723,0.0
8,46925.0,-2.604868,1.484446,0.470275,-1.247187,-0.596735,-1.435678,-0.042498,0.675088,0.670726,...,0.093393,-0.250085,-0.498523,-0.064745,0.615395,-0.345480,0.637373,0.212102,0.007527,0.0
9,145765.0,-0.769016,0.497769,-1.491102,-0.484560,1.974915,-1.546615,1.086798,-0.020257,-0.960554,...,-0.070164,0.333417,0.641867,-0.673254,-0.908480,1.080143,0.786867,-0.239764,-0.265091,0.0


In [41]:
newobs = pred.to_dict(orient = 'records')

In [48]:
newobs[0]

{'Time': 140315.0,
 'V1': 1.88275144365981,
 'V2': 0.5321745574787371,
 'V3': 0.150708356622915,
 'V4': 3.8894693444449104,
 'V5': 0.0110367165296031,
 'V6': 0.10161692045463001,
 'V7': -0.105079317724247,
 'V8': -0.0276783367013348,
 'V9': -0.649422331552985,
 'V10': 1.2760161964761398,
 'V11': -1.0975780754147801,
 'V12': 0.7253332584452471,
 'V13': 1.30091293887403,
 'V14': -0.404936212520168,
 'V15': -1.26229079100372,
 'V16': 0.872501700920738,
 'V17': -0.762329639638684,
 'V18': -0.954372139279484,
 'V19': -1.26536709127823,
 'V20': -0.22555835884440698,
 'V21': -0.401006076766849,
 'V22': -1.01558064776831,
 'V23': 0.487634612397945,
 'V24': -0.0242502150262368,
 'V25': -0.512936399003697,
 'V26': -0.51604963380311,
 'V27': 0.0111793306485451,
 'V28': -0.0266076514394325,
 'Amount': 0.0}

In [49]:
from typing import Dict, List, Union

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    client_options = {"api_endpoint": api_endpoint}
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    instances = instances if isinstance(instances, list) else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))


In [51]:
predict_custom_trained_model_sample(
    project="993073267534",
    endpoint_id="5892812777856172032",
    location="us-central1",
    instances=newobs[0]
)

response
 deployed_model_id: 5904164135901331456
 prediction: {'predicted_Class': '0', 'Class_values': ['1', '0'], 'Class_probs': [0.1213923841714859, 0.8786076307296753]}
